In [30]:
import os, tarfile, urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data():
    os.makedirs(SPAM_PATH, exist_ok=True)
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        path = os.path.join(SPAM_PATH, filename)
        urllib.request.urlretrieve(url, path)
        with tarfile.open(path) as t:
            t.extractall(SPAM_PATH)

fetch_spam_data()


/tmp/ipykernel_5747/1514786103.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(SPAM_PATH)


In [31]:
import email, email.policy

def load_email(is_spam, filename, base=SPAM_PATH):
    folder = "spam" if is_spam else "easy_ham"
    path = os.path.join(base, folder, filename)
    with open(path, "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

ham_files = os.listdir(os.path.join(SPAM_PATH, "easy_ham"))
spam_files = os.listdir(os.path.join(SPAM_PATH, "spam"))

ham_emails = [load_email(False, fn) for fn in ham_files if len(fn) > 20]
spam_emails = [load_email(True, fn) for fn in spam_files if len(fn) > 20]


In [33]:
def get_body(msg):
    parts = [msg] if msg.get_content_type() == 'text/plain' else msg.walk()
    for part in parts:
        if part.get_content_type() == 'text/plain':
            try:
                payload = part.get_payload(decode=True)
                charset = part.get_content_charset()
                if charset:
                    return payload.decode(charset, errors='replace').strip()
                else:
                    return payload.decode('utf-8', errors='replace').strip()
            except:
                return ""  # fallback in case decoding fails
    return ""


NameError: name 'string' is not defined